# DATA INTEGRATION (API DATA AND WEBSCRAPING DATA)

In [ ]:
# import libraries
import pandas as pd
import re
from rapidfuzz import process, fuzz
import unicodedata

# DATA PREPARATION

<br>

WEB SCRAPING DATA (FANTACALCIO.IT)
<br>
<br>

In [ ]:

file_path_scraping = '../1 - DataAcquisition/raw_data/df_players_grades.csv'
df_players_grades_original = pd.read_csv(file_path_scraping)

df_players_grades_original

,player_name,normalized_player_name,team_name,role,grade,fantagrade,goals_scored,goals_conceded,autogoals,penalties_scored,penalties_missed,penalties_saved,assists,player_of_the_match,season,matchday
0,Musso,Musso,Atalanta,Goalkeeper,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
1,Zappacosta,Zappacosta,Atalanta,Defender,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
2,Djimsiti,Djimsiti,Atalanta,Defender,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,1
3,Kolasinac,Kolasinac,Atalanta,Defender,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
4,Zortea,Zortea,Atalanta,Defender,7.0,10.0,1,0,0,0,0,0,0,0,2023/24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,Dani Silva,Dani Silva,Verona,Midfielder,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,38
11898,Mitrovic S.,S. Mitrovic,Verona,Midfielder,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,38
11899,Cisse' A.,A. Cisse',Verona,Midfielder,55.0,55.0,0,0,0,0,0,0,0,0,2023/24,38
11900,Noslin,Noslin,Verona,Attacker,7.5,11.5,1,0,0,0,0,0,1,0,2023/24,38


In [ ]:
# order the df by the column 'matchday'
df_players_grades = df_players_grades_original.sort_values(by='matchday')
# perform a groupby on 'player_name' and take the last value of 'team_name' for each group
df_players_grades = df_players_grades.groupby('player_name').agg({'normalized_player_name': 'last','team_name': 'last'}).reset_index()
# add a temporary index
df_players_grades['temporary_player_id'] = df_players_grades.index
# reorder the columns
df_players_grades = df_players_grades[['temporary_player_id', 'player_name', 'normalized_player_name', 'team_name']]

df_players_grades

,temporary_player_id,player_name,normalized_player_name,team_name
0,0,Abraham,Abraham,Roma
1,1,Acerbi,Acerbi,Inter
2,2,Adli,Adli,Milan
3,3,Adopo,Adopo,Atalanta
4,4,Aebischer,Aebischer,Bologna
...,...,...,...,...
582,582,Zielinski,Zielinski,Napoli
583,583,Zima,Zima,Torino
584,584,Zirkzee,Zirkzee,Bologna
585,585,Zortea,Zortea,Frosinone


<br>

API DATA
<br>
<br>

In [ ]:
file_path_api_players = '../1 - DataAcquisition/raw_data/all_players.csv'
file_path_api_teams = '../1 - DataAcquisition/raw_data/serie_a_teams_2024_25.csv'

df_api_players = pd.read_csv(file_path_api_players, index_col=0)
df_api_teams = pd.read_csv(file_path_api_teams, index_col=0)

In [ ]:
# rename the player name and the team name columns
df_api_players = df_api_players.rename(columns={'name': 'player_name'})
df_api_teams = df_api_teams.rename(columns={'name': 'team_name'})


df_api = pd.merge(df_api_players[['player_id', 'player_name', 'team_id']],
                  df_api_teams[['team_id', 'team_name']],
                  how='left', on='team_id').drop(columns='team_id')
df_api

,player_id,player_name,team_name
0,105,F. Ballo-Touré,AC Milan
1,305,D. Origi,AC Milan
2,765,A. Mirante,AC Milan
3,2045,S. Kjær,AC Milan
4,31054,R. Krunić,AC Milan
...,...,...,...
930,30912,A. Berardi,Verona
931,30922,D. Faraoni,Verona
932,31383,S. Perilli,Verona
933,125715,F. Daniliuc,Verona


In [ ]:
# team prefixes removal
def clean_team_name(team_name):
    return re.sub(r'^[A-Z]{1,3}\s', '', team_name)

df_api['team_name_cleaned'] = df_api['team_name'].apply(clean_team_name)
df_api['team_name_cleaned'].unique()

array(['Milan', 'Roma', 'Atalanta', 'Bologna', 'Cagliari', 'Como',
       'Empoli', 'Fiorentina', 'Genoa', 'Inter', 'Juventus', 'Lazio',
       'Lecce', 'Monza', 'Napoli', 'Parma', 'Torino', 'Udinese',
       'Venezia', 'Verona'], dtype=object)

<br>

WEB SCRAPING DATA (TRANSFERMARKT.IT)
<br>
<br>

In [ ]:
file_path_api_status_players = '../1 - DataAcquisition/raw_data/df_all_status_players_2024_25.csv'
df_api_status_players = pd.read_csv(file_path_api_status_players)

df_api_status_players

,player_id,player_status
0,105,Milan second team
1,305,Milan second team
2,765,Team of another league or released or retired
3,2045,Team of another league or released or retired
4,31054,Team of another league or released or retired
...,...,...
930,30912,Verona
931,30922,Verona
932,31383,Verona
933,125715,Verona


In [ ]:
# condition needed to take only the correct players
conditions = df_api_status_players['player_status'].isin(['Team of another league or released or retired']) | \
             df_api_status_players['player_status'].str.contains('second team|youth team', case=False, na=False)

player_ids = df_api_status_players.loc[conditions, 'player_id'].tolist()

In [ ]:
# filter the API source, keeping only the players who are playing in Serie A this year
df_api = df_api[~df_api['player_id'].isin(player_ids)]
df_api = df_api.drop(columns='team_name')

df_api

,player_id,player_name,team_name_cleaned
18,1696,S. Chukwueze,Milan
19,1828,L. Jović,Milan
21,19194,T. Abraham,Milan
22,22236,Rafael Leão,Milan
23,22254,Y. Fofana,Milan
...,...,...,...
929,30611,L. Montipò,Verona
930,30912,A. Berardi,Verona
931,30922,D. Faraoni,Verona
932,31383,S. Perilli,Verona


<br>

NAMES STANDARDIZATION
<br>
<br>

In [ ]:
def add_apostrophe_to_final_accented_vowel(text):
    # set of vowels with grave or acute accents
    accented_vowels = {'à', 'á', 'è', 'é', 'ì', 'í', 'ò', 'ó', 'ù', 'ú'}
    words = text.split()

    # function to add an apostrophe after an accented vowel
    def process_word(word):
        if word[-1] in accented_vowels:
            return f'{word}\''
        return word

    # apply the function to each word
    processed_words = [process_word(word) for word in words]

    return ' '.join(processed_words)

In [ ]:
def diacritic_removal(text):
    # NFD decomposition
    normalized = unicodedata.normalize('NFD', text)

    return ''.join([char for char in normalized if not unicodedata.combining(char)])

In [ ]:
def normalize_special_letters(text):
    # conversion map of special and combined letters
    special_letter_map = {
        'æ': 'ae', 'Æ': 'Ae',
        'œ': 'oe', 'Œ': 'Oe',
        'ß': 'ss',
        'Ĳ': 'IJ', 'ĳ': 'ij',
        'Đ': 'D', 'đ': 'd',
        'Ł': 'L', 'ł': 'l',
        'Ø': 'O', 'ø': 'o',
        'Ŋ': 'N', 'ŋ': 'n'
    }

    for letter, replacement in special_letter_map.items():
        text = text.replace(letter, replacement)

    # remove any diacritics with NFD normalization
    normalized = unicodedata.normalize('NFD', text)
    text_without_special_letters = ''.join([char for char in normalized if not unicodedata.combining(char)])

    return text_without_special_letters

In [ ]:
def name_normalization(row):
    # normalization pipeline
    text = row['player_name']
    text = add_apostrophe_to_final_accented_vowel(text)
    text = diacritic_removal(text)
    text = normalize_special_letters(text)
    return text

In [ ]:
df_api['normalized_player_name'] = df_api.apply(name_normalization, axis=1)
df_api = df_api[['player_id', 'player_name', 'normalized_player_name', 'team_name_cleaned']]

df_api

,player_id,player_name,normalized_player_name,team_name_cleaned
18,1696,S. Chukwueze,S. Chukwueze,Milan
19,1828,L. Jović,L. Jovic,Milan
21,19194,T. Abraham,T. Abraham,Milan
22,22236,Rafael Leão,Rafael Leao,Milan
23,22254,Y. Fofana,Y. Fofana,Milan
...,...,...,...,...
929,30611,L. Montipò,L. Montipo',Verona
930,30912,A. Berardi,A. Berardi,Verona
931,30922,D. Faraoni,D. Faraoni,Verona
932,31383,S. Perilli,S. Perilli,Verona


# DATA INTEGRATION

In [ ]:
df_players_grades.loc[:, 'player_id_matched'] = None
df_api.loc[:, 'player_id_matched'] = None

common_teams = set(df_api['team_name_cleaned']).intersection(set(df_players_grades['team_name']))
common_teams

C:\Users\andre\AppData\Local\Temp\ipykernel_30332\4014844805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api.loc[:, 'player_id_matched'] = None


{'Atalanta',
 'Bologna',
 'Cagliari',
 'Empoli',
 'Fiorentina',
 'Genoa',
 'Inter',
 'Juventus',
 'Lazio',
 'Lecce',
 'Milan',
 'Monza',
 'Napoli',
 'Roma',
 'Torino',
 'Udinese',
 'Verona'}

<br>

STEP 1
<br>
<br>

In [ ]:
matching_results_01 = []

# iterate for each common team
for common_team in common_teams:
    # filter the players of that team in the two DataFrames
    players_df_api = df_api[df_api['team_name_cleaned'] == common_team]
    players_df_grades = df_players_grades[df_players_grades['team_name'] == common_team]

    # for each player in the first DataFrame (players_df_api), look for a match in the second DataFrame (players_df_grades)
    for api_idx, row_api in players_df_api.iterrows():
        player_name_api = row_api['normalized_player_name']

        # find the best match with a threshold of 80
        match = process.extractOne(player_name_api, players_df_grades['normalized_player_name'], scorer=fuzz.ratio, score_cutoff=80)

        # if a match exists, update 'player_id_matched'
        if match:
            matched_name, score, matched_idx = match

            matched_player = players_df_grades.loc[matched_idx]

            # add the result to the list for monitoring
            matching_results_01.append({
                'team_name': common_team,
                'player_id_grades': matched_player['temporary_player_id'],
                'player_name_grades': matched_name,
                'player_name_api': player_name_api,
                'player_id_api': row_api['player_id'],
                'matching_score': score
            })

# DataFrame to visualize the matching results
df_matches = pd.DataFrame(matching_results_01)

df_matches

,team_name,player_id_grades,player_name_grades,player_name_api,player_id_api,matching_score
0,Inter,163,Dimarco,F. Dimarco,31010,82.352941
1,Inter,176,Dumfries,D. Dumfries,226,84.210526
2,Inter,142,Darmian,M. Darmian,887,82.352941
3,Inter,370,Mkhitaryan,H. Mkhitaryan,1457,86.956522
4,Inter,86,Calhanoglu,H. Calhanoglu,1640,86.956522
...,...,...,...,...,...,...
214,Torino,488,Schuurs,P. Schuurs,535,82.352941
215,Torino,563,Vojvoda,M. Vojvoda,8586,82.352941
216,Torino,349,Masina,A. Masina,18799,80.000000
217,Torino,298,Lazaro,V. Lazaro,25353,80.000000


In [ ]:
print(len(df_matches['player_id_grades']), len(df_matches['player_id_grades'].unique()))

219 219


In [ ]:
print(len(df_matches['player_id_api']), len(df_matches['player_id_api'].unique()))

219 219


In [ ]:
# update df_players_grades and df_api
for _ , row in df_matches.iterrows():
    id_grades = row['player_id_grades']
    id_api = row['player_id_api']

    df_players_grades.loc[df_players_grades['temporary_player_id'] == id_grades, 'player_id_matched'] = id_api
    df_api.loc[df_api['player_id'] == id_api, 'player_id_matched'] = id_grades

<br>

STEP 2
<br>
<br>

In [ ]:
matching_results_02 = []

# iterate for each common team
for common_team in common_teams:
    # filter the players of that team in the two DataFrames (only the players not already matched)
    players_df_api = df_api[(df_api['team_name_cleaned'] == common_team) & (df_api['player_id_matched'].isnull())]
    players_df_grades = df_players_grades[(df_players_grades['team_name'] == common_team) & (df_players_grades['player_id_matched'].isnull())]

    # for each player in the first DataFrame (players_df_api), look for a match in the second DataFrame (players_df_grades)
    for api_idx, row_api in players_df_api.iterrows():
        player_name_api = row_api['normalized_player_name']

        # find the best match with a threshold of 74
        match = process.extractOne(player_name_api, players_df_grades['normalized_player_name'], scorer=fuzz.ratio, score_cutoff=74)

        # if a match exists, update 'player_id_matched'
        if match:
            matched_name, score, matched_idx = match

            matched_player = players_df_grades.loc[matched_idx]

            # add the result to the list for monitoring
            matching_results_02.append({
                'team_name': common_team,
                'player_id_grades': matched_player['temporary_player_id'],
                'player_name_grades': matched_name,
                'player_name_api': player_name_api,
                'player_id_api': row_api['player_id'],
                'matching_score': score
            })

# DataFrame to visualize the matching results
df_matches = pd.DataFrame(matching_results_02)

df_matches

,team_name,player_id_grades,player_name_grades,player_name_api,player_id_api,matching_score
0,Inter,346,L. Martinez,Lautaro Martinez,217,74.074074
1,Juventus,361,Milik,A. Milik,333,76.923077
2,Juventus,441,Pogba,P. Pogba,904,76.923077
3,Juventus,428,Perin,M. Perin,849,76.923077
4,Juventus,213,Gatti,F. Gatti,268341,76.923077
5,Napoli,358,Meret,A. Meret,312,76.923077
6,Udinese,316,Lucca,L. Lucca,199089,76.923077
7,Udinese,52,Bijol,J. Bijol,833,76.923077
8,Udinese,398,Okoye,M. Okoye,143648,76.923077
9,Milan,263,Jovic,L. Jovic,1828,76.923077


In [ ]:
print(len(df_matches['player_id_grades']), len(df_matches['player_id_grades'].unique()))

34 34


In [ ]:
# update df_players_grades and df_api
for _ , row in df_matches.iterrows():
    id_grades = row['player_id_grades']
    id_api = row['player_id_api']

    df_players_grades.loc[df_players_grades['temporary_player_id'] == id_grades, 'player_id_matched'] = id_api
    df_api.loc[df_api['player_id'] == id_api, 'player_id_matched'] = id_grades

In [ ]:
df_api[df_api['normalized_player_name']=='Josep Martinez']

,player_id,player_name,normalized_player_name,team_name_cleaned,player_id_matched
483,46988,Josep Martínez,Josep Martinez,Inter,None


In [ ]:
df_api[df_api['normalized_player_name']=='Lautaro Martinez']

,player_id,player_name,normalized_player_name,team_name_cleaned,player_id_matched
456,217,Lautaro Martínez,Lautaro Martinez,Inter,346


In [ ]:
df_players_grades[df_players_grades['normalized_player_name']=='L. Martinez']

,temporary_player_id,player_name,normalized_player_name,team_name,player_id_matched
346,346,Martinez L.,L. Martinez,Inter,217


<br>

STEP 3
<br>
<br>

In [ ]:
matching_results_03 = []

# iterate for each common team
for common_team in common_teams:
    # filter the players of that team in the two DataFrames (only the players not already matched)
    players_df_api = df_api[(df_api['team_name_cleaned'] == common_team) & (df_api['player_id_matched'].isnull())]
    players_df_grades = df_players_grades[(df_players_grades['team_name'] == common_team) & (df_players_grades['player_id_matched'].isnull())]

    # for each player in the first DataFrame (players_df_api), look for a match in the second DataFrame (players_df_grades)
    for api_idx, row_api in players_df_api.iterrows():
        player_name_api = row_api['normalized_player_name']

        # find the best match with a threshold of 70
        match = process.extractOne(player_name_api, players_df_grades['normalized_player_name'], scorer=fuzz.ratio, score_cutoff=70)

        # if a match exists, update 'player_id_matched'
        if match:
            matched_name, score, matched_idx = match

            matched_player = players_df_grades.loc[matched_idx]

            # add the result to the list for monitoring
            matching_results_03.append({
                'team_name': common_team,
                'player_id_grades': matched_player['temporary_player_id'],
                'player_name_grades': matched_name,
                'player_name_api': player_name_api,
                'player_id_api': row_api['player_id'],
                'matching_score': score
            })

# DataFrame to visualize the matching results
df_matches = pd.DataFrame(matching_results_03)

df_matches

,team_name,player_id_grades,player_name_grades,player_name_api,player_id_api,matching_score
0,Inter,149,De Vrij,S. de Vrij,194,70.588235
1,Juventus,356,Mckennie,W. McKennie,415,73.684211
2,Juventus,567,Weah,T. Weah,1138,72.727273
3,Napoli,573,Zambo Anguissa,F. Anguissa,3406,72.000000
4,Udinese,578,Zarraga,Oier Zarraga,182560,73.684211
5,Monza,167,Djuric,M. Duric,31692,71.428571
6,Monza,257,Izzo,A. Izzo,30501,72.727273
7,Atalanta,182,D.s. Ederson,Ederson,10097,73.684211
8,Atalanta,146,De Roon,M. de Roon,30432,70.588235
9,Atalanta,243,Hien,I. Hien,137976,72.727273


In [ ]:
print(len(df_matches['player_id_api']), len(df_matches['player_id_api'].unique()))

18 18


In [ ]:
# update df_players_grades and df_api
for _ , row in df_matches.iterrows():
    id_grades = row['player_id_grades']
    id_api = row['player_id_api']

    df_players_grades.loc[df_players_grades['temporary_player_id'] == id_grades, 'player_id_matched'] = id_api
    df_api.loc[df_api['player_id'] == id_api, 'player_id_matched'] = id_grades

<br>

STEP 4
<br>
<br>

In [ ]:
matching_results_04 = []

# iterate for each common team
for common_team in common_teams:
    # filter the players of that team in the two DataFrames (only the players not already matched)
    players_df_api = df_api[(df_api['team_name_cleaned'] == common_team) & (df_api['player_id_matched'].isnull())]
    players_df_grades = df_players_grades[(df_players_grades['team_name'] == common_team) & (df_players_grades['player_id_matched'].isnull())]

    # for each player in the first DataFrame (players_df_api), look for a match in the second DataFrame (players_df_grades)
    for api_idx, row_api in players_df_api.iterrows():
        player_name_api = row_api['normalized_player_name']

        # find the best match with a threshold of 50
        match = process.extractOne(player_name_api, players_df_grades['normalized_player_name'], scorer=fuzz.ratio, score_cutoff=50)

        # if a match exists, update 'player_id_matched'
        if match:
            matched_name, score, matched_idx = match

            matched_player = players_df_grades.loc[matched_idx]

            # add the result to the list for monitoring
            matching_results_04.append({
                'team_name': common_team,
                'player_id_grades': matched_player['temporary_player_id'],
                'player_name_grades': matched_name,
                'player_name_api': player_name_api,
                'player_id_api': row_api['player_id'],
                'matching_score': score
            })

# DataFrame to visualize the matching results
df_matches = pd.DataFrame(matching_results_04)

df_matches

,team_name,player_id_grades,player_name_grades,player_name_api,player_id_api,matching_score
0,Juventus,388,Nonge,N. Gonzalez,26315,50.000000
1,Juventus,570,Yildiz,K. Yıldız,339883,53.333333
2,Udinese,5,M. Ake',I. Toure',195893,50.000000
3,Milan,469,L. Romero,L. Torriani,386298,50.000000
4,Roma,308,D. Llorente,E. Le Fee,20638,50.000000
5,Roma,308,D. Llorente,M. Kone',22147,52.631579
6,Roma,380,N'Dicka,E. Ndicka,1807,62.500000
7,Monza,374,Mota,Dany Mota,30603,61.538462
8,Monza,508,A. Sorrentino,A. Petagna,30879,52.173913
9,Monza,97,V. Carboni,O. Forson,284242,52.631579


OK:

Lazio	219	Gila	Mario Gila	162952

Genoa	359	Messias	Junior Messias	56396

Juventus	570	Yildiz	K. Yıldız	339883

Monza	374	Mota	Dany Mota	30603

Monza	338	Mari'	Pablo Mari'	46792

Cagliari	23	Azzi	Paulo Azzi	56207

Roma	380	  N'Dicka 	E. Ndicka	1807

In [ ]:
filtered_df_matches = df_matches[df_matches['player_id_grades'].isin([219, 359, 570, 374, 338, 23, 380])]

In [ ]:
print(len(filtered_df_matches['player_id_grades']), len(filtered_df_matches['player_id_grades'].unique()))

7 7


In [ ]:
# update df_players_grades and df_api
for _ , row in filtered_df_matches.iterrows():
    id_grades = row['player_id_grades']
    id_api = row['player_id_api']

    df_players_grades.loc[df_players_grades['temporary_player_id'] == id_grades, 'player_id_matched'] = id_api
    df_api.loc[df_api['player_id'] == id_api, 'player_id_matched'] = id_grades

<br>

STEP 5
<br>
<br>

In [ ]:
matching_results_05 = []


# filter the players who have not yet been matched
players_df_api = df_api[df_api['player_id_matched'].isnull()]
players_df_grades = df_players_grades[df_players_grades['player_id_matched'].isnull()]

# for each player in the first DataFrame (players_df_api), look for a match in the second DataFrame (players_df_grades)
for api_idx, row_api in players_df_api.iterrows():
    player_name_api = row_api['normalized_player_name']

    # find the best match in the first DataFrame with a threshold of 80
    match = process.extractOne(player_name_api, players_df_grades['normalized_player_name'], scorer=fuzz.ratio, score_cutoff=80)

    # if a match exists, update 'player_id_matched'
    if match:
        matched_name, score, matched_idx = match

        matched_player = players_df_grades.loc[matched_idx]

        # add the result to the list for monitoring
        matching_results_05.append({
            'team_name_grades': matched_player['team_name'],
            'player_id_grades': matched_player['temporary_player_id'],
            'player_name_grades': matched_name,
            'player_name_api': player_name_api,
            'player_id_api': row_api['player_id'],
            'team_name_api': row_api['team_name_cleaned'],
            'matching_score': score
        })

# DataFrame to visualize the matching results
df_matches = pd.DataFrame(matching_results_05)

df_matches

,team_name_grades,player_id_grades,player_name_grades,player_name_api,player_id_api,team_name_api,matching_score
0,Roma,0,Abraham,T. Abraham,19194,Milan,82.352941
1,Bologna,477,Saelemaekers,A. Saelemaekers,1417,Roma,88.888889
2,Cagliari,495,Shomurodov,E. Shomurodov,53535,Roma,86.956522
3,Frosinone,510,Soule',M. Soule',323936,Roma,80.000000
4,Frosinone,74,Brescianini,M. Brescianini,1639,Atalanta,88.000000
...,...,...,...,...,...,...,...
72,Frosinone,236,Harroui,A. Harroui,37437,Verona,82.352941
73,Salernitana,70,Bradaric,D. Bradaric,14327,Verona,84.210526
74,Sassuolo,45,Berardi,A. Berardi,30912,Verona,82.352941
75,Fiorentina,193,Faraoni,D. Faraoni,30922,Verona,82.352941



NO:

Atalanta	175	E. Toure'	I. Toure'	195893	Udinese

Juventus	299	L. Cerri	A. Cerri	30570	Como

Sassuolo	45	Berardi	A. Berardi	30912	Verona

In [ ]:
filtered_df_matches = df_matches[~df_matches['player_id_grades'].isin([175, 299, 45])]

In [ ]:
print(len(filtered_df_matches['player_id_grades']), len(filtered_df_matches['player_id_grades'].unique()))

76 74


In [ ]:
duplicates = filtered_df_matches['player_id_grades'][filtered_df_matches['player_id_grades'].duplicated()]

duplicate_rows = filtered_df_matches[filtered_df_matches['player_id_grades'].isin(duplicates)]

duplicate_rows

,team_name_grades,player_id_grades,player_name_grades,player_name_api,player_id_api,team_name_api,matching_score
40,Fiorentina,227,N. Gonzalez,N. Gonzalez,26315,Juventus,100.000000
47,Fiorentina,227,N. Gonzalez,D. Gonzalez,307103,Lazio,90.909091
49,Salernitana,131,L. Coulibaly,L. Coulibaly,1748,Lecce,100.000000
63,Salernitana,131,L. Coulibaly,W. Coulibaly,128338,Parma,91.666667


In [ ]:
filtered_df_matches = filtered_df_matches[~filtered_df_matches['player_id_api'].isin([307103, 128338])]

In [ ]:
print(len(filtered_df_matches['player_id_grades']), len(filtered_df_matches['player_id_grades'].unique()))

74 74


In [ ]:
print(len(filtered_df_matches['player_id_api']), len(filtered_df_matches['player_id_api'].unique()))

74 74


In [ ]:
# update df_players_grades and df_api
for _ , row in filtered_df_matches.iterrows():
    id_grades = row['player_id_grades']
    id_api = row['player_id_api']

    df_players_grades.loc[df_players_grades['temporary_player_id'] == id_grades, 'player_id_matched'] = id_api
    df_api.loc[df_api['player_id'] == id_api, 'player_id_matched'] = id_grades

<br>

STEP 6
<br>
<br>

In [ ]:
matching_results_06 = []


# filter the players who have not yet been matched
players_df_api = df_api[(df_api['player_id_matched'].isnull())]
players_df_grades = df_players_grades[df_players_grades['player_id_matched'].isnull()]

# for each player in the first DataFrame (players_df_api), look for a match in the second DataFrame (players_df_grades)
for api_idx, row_api in players_df_api.iterrows():
    player_name_api = row_api['normalized_player_name']

    # find the best match in the first DataFrame with a threshold of 72
    match = process.extractOne(player_name_api, players_df_grades['normalized_player_name'], scorer=fuzz.ratio, score_cutoff=72)

    # if a match exists, update 'player_id_matched'
    if match:
        matched_name, score, matched_idx = match

        matched_player = players_df_grades.loc[matched_idx]

        # add the result to the list for monitoring
        matching_results_06.append({
            'team_name_grades': matched_player['team_name'],
            'player_id_grades': matched_player['temporary_player_id'],
            'player_name_grades': matched_name,
            'player_name_api': player_name_api,
            'player_id_api': row_api['player_id'],
            'team_name_api': row_api['team_name_cleaned'],
            'matching_score': score
        })
# DataFrame to visualize the matching results
df_matches = pd.DataFrame(matching_results_06)
df_matches = df_matches[df_matches['matching_score']<80]

df_matches

,team_name_grades,player_id_grades,player_name_grades,player_name_api,player_id_api,team_name_api,matching_score
0,Torino,507,Soppy,B. Soppy,162347,Atalanta,76.923077
1,Sassuolo,365,K. Miranda,Juan Miranda,134,Bologna,72.727273
2,Sassuolo,189,Erlic,M. Erlic,30827,Bologna,76.923077
3,Atalanta,244,Holm,E. Holm,47985,Bologna,72.727273
4,Empoli,340,Marin,R. Marin,2118,Cagliari,76.923077
5,Atalanta,3,Adopo,M. Adopo,30505,Cagliari,76.923077
6,Torino,551,Verdi,S. Verdi,334,Como,76.923077
7,Sassuolo,557,Viti,M. Viti,180510,Empoli,72.727273
8,Juventus,277,Kean,M. Kean,877,Fiorentina,72.727273
9,Milan,2,Adli,Y. Adli,1265,Fiorentina,72.727273


NO:

Sassuolo	365	K. Miranda	Juan Miranda	134	Bologna

Empoli	496	S. Shpendi	S. Sensi	30535	Monza

Milan	87	Camarda	D. Camara	91424	Parma

In [ ]:
filtered_df_matches = df_matches[~df_matches['player_id_grades'].isin([365, 496, 87])]

In [ ]:
print(len(filtered_df_matches['player_id_grades']), len(filtered_df_matches['player_id_grades'].unique()))

13 13


In [ ]:
# update df_players_grades and df_api
for _ , row in filtered_df_matches.iterrows():
    id_grades = row['player_id_grades']
    id_api = row['player_id_api']

    df_players_grades.loc[df_players_grades['temporary_player_id'] == id_grades, 'player_id_matched'] = id_api
    df_api.loc[df_api['player_id'] == id_api, 'player_id_matched'] = id_grades

<br>

STEP 7
<br>
<br>

In [ ]:
matching_results_07 = []


# filter the players who have not yet been matched
players_df_api = df_api[(df_api['player_id_matched'].isnull()) & (~df_api['player_id'].isin([134, 30535, 91424]))]
players_df_grades = df_players_grades[df_players_grades['player_id_matched'].isnull()]

# for each player in the first DataFrame (players_df_api), look for a match in the second DataFrame (players_df_grades)
for api_idx, row_api in players_df_api.iterrows():
    player_name_api = row_api['normalized_player_name']

    # find the best match in the first DataFrame with a threshold of 60
    match = process.extractOne(player_name_api, players_df_grades['normalized_player_name'], scorer=fuzz.ratio, score_cutoff=60)

    # if a match exists, update 'player_id_matched'
    if match:
        matched_name, score, matched_idx = match

        matched_player = players_df_grades.loc[matched_idx]

        # add the result to the list for monitoring
        matching_results_07.append({
            'team_name_grades': matched_player['team_name'],
            'player_id_grades': matched_player['temporary_player_id'],
            'player_name_grades': matched_name,
            'player_name_api': player_name_api,
            'player_id_api': row_api['player_id'],
            'team_name_api': row_api['team_name_cleaned'],
            'matching_score': score
        })

# DataFrame to visualize the matching results
df_matches = pd.DataFrame(matching_results_07)
df_matches = df_matches[df_matches['matching_score'] <= 72]

OK:

Lecce	515	Strefezza	Gabriel Strefezza	31624	Como

Genoa	343	Martin	Aaron Martin	25914	Genoa

Salernitana	162	Dia	B. Dia	22015.0	Lazio

Fiorentina	18	Arthur Melo	  Arthur	150	Juventus

Juventus	387	Nicolussi Caviglia	H. Nicolussi	881	Venezia

In [ ]:
filtered_df_matches = df_matches[(df_matches['player_id_grades'].isin([515, 343, 162, 18, 387]))]

In [ ]:
print(len(filtered_df_matches['player_id_grades']), len(filtered_df_matches['player_id_grades'].unique()))

7 5


In [ ]:
filtered_df_matches

,team_name_grades,player_id_grades,player_name_grades,player_name_api,player_id_api,team_name_api,matching_score
8,Lecce,515,Strefezza,Gabriel Strefezza,31624,Como,69.230769
17,Genoa,343,Martin,Aaron Martin,25914,Genoa,66.666667
20,Fiorentina,18,Arthur Melo,Arthur,150,Juventus,70.588235
23,Salernitana,162,Dia,B. Dia,22015,Lazio,66.666667
33,Salernitana,162,Dia,D. Diaw,30663,Monza,60.000000
34,Genoa,343,Martin,Rafa Marin,237066,Napoli,62.500000
46,Juventus,387,Nicolussi Caviglia,H. Nicolussi,881,Venezia,60.000000


In [ ]:
filtered_df_matches = filtered_df_matches[~filtered_df_matches['player_id_api'].isin([30663, 237066])]

In [ ]:
print(len(filtered_df_matches['player_id_grades']), len(filtered_df_matches['player_id_grades'].unique()))

5 5


In [ ]:
print(len(filtered_df_matches['player_id_api']), len(filtered_df_matches['player_id_api'].unique()))

5 5


In [ ]:
# update df_players_grades and df_api
for _ , row in filtered_df_matches.iterrows():
    id_grades = row['player_id_grades']
    id_api = row['player_id_api']

    df_players_grades.loc[df_players_grades['temporary_player_id'] == id_grades, 'player_id_matched'] = id_api
    df_api.loc[df_api['player_id'] == id_api, 'player_id_matched'] = id_grades

<br>

STEP 8
<br>
<br>

In [ ]:
matching_results_08 = []


# filter the players who have not yet been matched
players_df_api = df_api[(df_api['player_id_matched'].isnull()) & (~df_api['player_id'].isin([134, 30535, 91424]))]
players_df_grades = df_players_grades[df_players_grades['player_id_matched'].isnull()]

# for each player in the first DataFrame (players_df_api), look for a match in the second DataFrame (players_df_grades)
for api_idx, row_api in players_df_api.iterrows():
    player_name_api = row_api['normalized_player_name']

    # find the best match in the first DataFrame with a threshold of 60
    match = process.extractOne(player_name_api, players_df_grades['normalized_player_name'], scorer=fuzz.ratio, score_cutoff=50)

    # if a match exists, update 'player_id_matched'
    if match:
        matched_name, score, matched_idx = match

        matched_player = players_df_grades.loc[matched_idx]

        # add the result to the list for monitoring
        matching_results_08.append({
            'team_name_grades': matched_player['team_name'],
            'player_id_grades': matched_player['temporary_player_id'],
            'player_name_grades': matched_name,
            'player_name_api': player_name_api,
            'player_id_api': row_api['player_id'],
            'team_name_api': row_api['team_name_cleaned'],
            'matching_score': score
        })

# DataFrame to visualize the matching results
df_matches = pd.DataFrame(matching_results_08)
df_matches = df_matches[df_matches['matching_score'] <= 60]

<br>

NOT MATCHED PLAYERS
<br>
<br>

In [ ]:
len(df_players_grades[df_players_grades['player_id_matched'].isnull()])

217

In [ ]:
len(df_api[df_api['player_id_matched'].isnull()])

211

# NAME HARMONIZATION

In [ ]:
df_players_grades_original['api_player_id'] = None

# filter the df (only matched players)
df_players_grades_not_nulls = df_players_grades[~df_players_grades['player_id_matched'].isnull()]

for idx, row in df_players_grades_not_nulls.iterrows():
    # player name and team name (from df_players_grades source) extraction
    player_name = row['player_name']
    team_name = row['team_name']

    # player name (from df_api source) extraction
    player_id_matched = row['player_id_matched']
    condition = df_api['player_id']==player_id_matched
    api_player_name = df_api.loc[condition, 'player_name'].iloc[0]

    # substitution of the name in df_players_grades_original
    conditions = (df_players_grades_original['player_name'] == player_name)

    df_players_grades_original.loc[conditions, 'player_name'] = api_player_name
    df_players_grades_original.loc[conditions, 'api_player_id'] = player_id_matched

df_players_grades_original = df_players_grades_original[['api_player_id', 'player_name',
                                                         'normalized_player_name', 'team_name', 'role',
                                                         'grade', 'fantagrade', 'goals_scored',
                                                         'goals_conceded', 'autogoals', 'penalties_scored',
                                                         'penalties_missed', 'penalties_saved', 'assists',
                                                         'player_of_the_match', 'season', 'matchday']]

df_players_grades_original

,api_player_id,player_name,normalized_player_name,team_name,role,grade,fantagrade,goals_scored,goals_conceded,autogoals,penalties_scored,penalties_missed,penalties_saved,assists,player_of_the_match,season,matchday
0,None,Musso,Musso,Atalanta,Goalkeeper,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
1,2286,D. Zappacosta,Zappacosta,Atalanta,Defender,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
2,30421,B. Djimsiti,Djimsiti,Atalanta,Defender,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,1
3,1442,S. Kolašinac,Kolasinac,Atalanta,Defender,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
4,128461,N. Zortea,Zortea,Atalanta,Defender,7.0,10.0,1,0,0,0,0,0,0,0,2023/24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,42007,Dani Silva,Dani Silva,Verona,Midfielder,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,38
11898,None,Mitrovic S.,S. Mitrovic,Verona,Midfielder,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,38
11899,359100,A. Cissè,A. Cisse',Verona,Midfielder,55.0,55.0,0,0,0,0,0,0,0,0,2023/24,38
11900,133729,T. Noslin,Noslin,Verona,Attacker,7.5,11.5,1,0,0,0,0,0,1,0,2023/24,38


In [ ]:
len(set(df_players_grades_original['api_player_id']) - set([None]))

370

In [ ]:
df_players_grades_original.to_csv('../1 - DataAcquisition/raw_data/df_players_grades_post_integration.csv',
                                  index=False)

In [ ]:
df_players_grades_original

,api_player_id,player_name,normalized_player_name,team_name,role,grade,fantagrade,goals_scored,goals_conceded,autogoals,penalties_scored,penalties_missed,penalties_saved,assists,player_of_the_match,season,matchday
0,None,Musso,Musso,Atalanta,Goalkeeper,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
1,2286,D. Zappacosta,Zappacosta,Atalanta,Defender,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
2,30421,B. Djimsiti,Djimsiti,Atalanta,Defender,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,1
3,1442,S. Kolašinac,Kolasinac,Atalanta,Defender,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
4,128461,N. Zortea,Zortea,Atalanta,Defender,7.0,10.0,1,0,0,0,0,0,0,0,2023/24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,42007,Dani Silva,Dani Silva,Verona,Midfielder,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,38
11898,None,Mitrovic S.,S. Mitrovic,Verona,Midfielder,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,38
11899,359100,A. Cissè,A. Cisse',Verona,Midfielder,55.0,55.0,0,0,0,0,0,0,0,0,2023/24,38
11900,133729,T. Noslin,Noslin,Verona,Attacker,7.5,11.5,1,0,0,0,0,0,1,0,2023/24,38
